# Bayesian Modelling with Lumen

This notebook showcases how the Lumen project can be utilized for Bayesian modelling.

Bayesian modelling can be done with Probabilistic Programming Languages, such as PyMC3 or Stan.
Lumen neatly integrated with PyMC3 (and Stan, which is under development), that is it you export your
PyMC3 model to Lumen with a single call. Once exported, you can visually explore, debug, and validate your model.
 No further coding required!

## Bayesian Modeling Overview

Bayesian Modelling can be separated into the following stages.

  0. data collection:
  1. data exploration: understand existing structure in your data, find issues with data quality, develop hypothesis
  how to structure your model to solve your problem
  2. model setup: describe model in term of a PPL
  3. model validation: check if the model does what you want and expect it to do.
  4. model deployment:

Typically this is an very incremental workflow where at first you start with a simple model which
thereafter is improved by changing the model, it's underlying distributions, by incooperating more data,
by differently perprocessing the data, ...

Often times there actually is not observable data available, hence the steps involving data exploratino and validation
with respect to traning or test data become obsolete. However, there is still the need to validate, debug and improve
the model - tasks with which Lumen may support you.

## How does Lumen help with Bayesian Modeling

Lumen allows you to:

   * export a PyMC3 model to Lumen in a single call
   * visually and interactively explore models:
      * understand prior and posterior distribution (that is, of the paramters/latent variables)
      * understand prior predictive and posterior predictive distribution (that is of the observable variables)
      * compare multiple increments of a model side by side
      * compare model distribution with data distribution (training or test data)
      * use Posterior Predictive Checks for validation


## Overview on Lumen

Lumen consists of a backend and a front end part.

### Backend
The backend serves two purposes:
 1. manage models, watch a specific directory for new models/changed models, and
 2. provide an API to run complex inference queries on models.

You may use the API directly (see also: TODO) to run inference queries, however, in many
cases it maybe much more convenient to use the front-end instead.
If you wonder what queries are, then you may imagine them as specific questions, that you ask the model.
Here are some examples:

  * 'How does that marginal distribution of the variable "age" look like?'
  * 'What is the most likely value for "income" given that a person has "low education"?'
  * 'What do samples drawn from the model look like for the variables "east-west" and "age"?

### Frontend
The front-end gives you a visual interactive interface to configure, run and visualize a wide
range of possibly complex queries.
It does not require any programming from your side. The front-end connects
to an instance of the backend to actually execute any queries.

## Example Modelling Workflow

The following shows a simple step-by-step example how a model is iteratively refined, and how each iterate
analyzed with the Lumen frontend to validate and debug it.

A precondition is that you have both frontend and backend installed on your machine.

_last time tested: 2020-09-22_

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pymc3 as pm
import mb_modelbase as mb

# folder in which we will store models
models_path = './models_bayesian_modeling_example'
import os
if not os.path.exists(models_path):
    os.makedirs(models_path)

### Start Lumen

To work with Lumen we simply have to:
 1. start the back-end,
 2. start the front-end, and
 3. export each PyMC3 model increment and save it in the folder `models_path` so that we can use it with Lumen.

#### Back-End
The backend watches for changes in a folder.
Run the following on a separate console to start the backend and let it watch models in the specified folder:

```
cd <dir-where-you-cloned-the-backend-source-to>
python3 bin/webservice.py --d jupyter/models_bayesian_modeling_example
```

#### Front-End
The front-end is by default configured to use a local backend, that is, you don't have to do anything, but run it.
Simply open its `index.html` with a browser (preferably, chrome/chromium based).

Now, backend and frontend are ready. Let's start with the modelling workflow and create some models... !

### Collect data

Here, we simply create some fake observed data:

In [2]:
np.random.seed(123)
alpha, sigma = 1, 1
beta_0 = 1
beta_1 = 2.5
size = 100
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2
Y = alpha + beta_0 * X1 + beta_1 * X2 + np.random.randn(size) * sigma
data = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y})

### Exploration of observed data

Let's use Lumen to explore our observed data. To do so, we have to wrap it in a model and save it our models directory.
Here, we use a simple histogram estimator. You may also use a Kernel-Density-Estimator by changing the parameter
`model_type` to 'kde'.

In [3]:
model_empirical = mb.make_empirical_model(modelname='data_empirical', output_directory=models_path, df=data, model_type='empirical')
#model_kde = mb.make_empirical_model(modelname='data_kde', output_directory=models_path, df=data, model_type='kde')

After execution of the previous cell, have a look at Lumen. It should inform you that a new model was found:

![Screenhot](img/new_model_found.png)

You can now explore it! Have fun ;)

### Creation of initial PyMC3 model

Let's now create a very basic first model using PyMC3. This illustrates how the wrapping/the export to Lumen is done.

In [4]:
model_name = "model_1"
model_1 = pm.Model()

with model_1:
    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta_0 = pm.Normal('beta_0', mu=0, sd=10)
    beta_1 = pm.Normal('beta_1', mu=0, sd=20)
    sigma = pm.HalfNormal('sigma', sd=5)
    
    # Expected value of outcome
    mu = alpha + beta_0 * data['X1'] + beta_1 * data['X2']
    
    # Likelihood (sampling distribution) of observations
    Y = pm.Normal('Y', mu=mu, sd=sigma, observed=data['Y'])
    X1 = pm.Normal('X1', mu=data['X1'], sd=sigma, observed=data['X1'])
    X2 = pm.Normal('X2', mu=data['X2'], sd=sigma, observed=data['X2'])

    ## wrap PyMC3 model with Lumen!
    model = mb.ProbabilisticPymc3Model(model_name, model_1)
    model.fit(data)

## Save model the folder to that it shows up in front-end
model_1.save(models_path)

Auto-assigning NUTS sampler...
15:06:40.282 INFO :: Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
15:06:40.284 INFO :: Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
15:06:42.631 INFO :: Sequential sampling (1 chains in 1 job)
NUTS: [sigma, beta_1, beta_0, alpha]
15:06:42.633 INFO :: NUTS: [sigma, beta_1, beta_0, alpha]
Only one chain was sampled, this makes it impossible to run some convergence checks
15:06:47.260 INFO :: Only one chain was sampled, this makes it impossible to run some convergence checks
100%|██████████| 5000/5000 [00:13<00:00, 384.29it/s]


AttributeError: 'Model' object has no attribute 'save'

The initial model pops up in the front-end and is available for exploration:

![Screenhot](img/new_model_found_2.png)

We easily see, for instance, that the resulting posterior predictive distributions of X1 and X2 do not fit well
with the observed empirical distributions:

![Screenhot](img/model1_misfit.png)

We could now improve the model and check the new increment ... :)

In [11]:
# TODO: make a new model
# create modified PPL model with PyMC3
model_2 = pm.Model()
model_name = "model_2"

with model_2:
    # Priors for unknown model parameters
    alpha = pm.Uniform('alpha')
    beta_0 = pm.Normal('beta_0', mu=20, sd=1)
    beta_1 = pm.unIF('beta_1', mu=5, sd=1)
    sigma = pm.HalfNormal('sigma', sd=1)
    
    # Expected value of outcome
    mu = alpha + beta_0 * data['X1'] + beta_1 * data['X2']
    
    # Likelihood (sampling distribution) of observations
    Y = pm.Normal('Y', mu=mu, sd=sigma, observed=data['Y'])
    X1 = pm.Normal('X1', mu=data['X1'], sd=sigma, observed=data['X1'])
    X2 = pm.Normal('X2', mu=data['X2'], sd=sigma, observed=data['X2'])
    
    model = mb.ProbabilisticPymc3Model(model_name, model_2)
    model.fit(data)
    
# we could reuse the empirical model of model_1. However, this only works if the data didn't change.
#model_2.set_empirical_model_name(model_1.get_empiricial_model_name())

model_2.save(models_path)

AttributeError: module 'pymc3' has no attribute 'unIF'